# 日本語 BERT Base Model Fine-tuning & Deployment on Inferentia2
本 Notebook の元ネタのブログはこちらから
+ https://aws.amazon.com/jp/blogs/news/aws-trainium-amazon-ec2-trn1-ml-training-part1/
+ https://aws.amazon.com/jp/blogs/news/aws-trainium-amazon-ec2-trn1-ml-training-part2/

## 事前準備

In [ ]:
%env TOKENIZERS_PARALLELISM=True #Supresses tokenizer warnings making errors easier to detect
!pip install -U pip
!pip install -U transformers[ja]==4.52.3 datasets accelerate

In [ ]:
!pip list | grep "neuron\|torch"

In [ ]:
!sudo rmmod neuron; sudo modprobe neuron

## データセットの準備
本テストでは、Huggingface Hub で利用可能な以下のセンチメント（感情）データセットのうち、日本語のサブセットを使用します。
https://huggingface.co/datasets/tyqiangz/multilingual-sentiments

本テストではテキストデータをPositiveかNegativeに分類する 2 クラスの分類問題として扱うことにします。元々のデータセットは positive(LABEL_0)、neutral(LABEL_1)、negative(LABEL_2)としてラベル付けされていますが、neutralのデータは使用しないこととし、ラベルをpositive(LABEL_0)、negative(LABEL_1)として再定義します。

In [ ]:
from transformers import BertJapaneseTokenizer
from datasets import load_dataset

# Prepare dataset
dataset = load_dataset("tyqiangz/multilingual-sentiments", "japanese")
print(dataset)

print(dataset["train"].features)

dataset = dataset.remove_columns(["source"])
dataset = dataset.filter(lambda dataset: dataset["label"] != 1)
dataset = dataset.map(lambda dataset: {"labels": int(dataset["label"] == 2)}, remove_columns=["label"])

print(dataset["train"][20000])
print(dataset["train"][50000])

次に、文章テキストのままだとモデルのトレーニングはできないため、テキストを意味のある単位で分割（トークナイズ）した上で数値に変換します。トークナイザーには MeCab ベースの BertJapaneseTokenizer を利用しました。

In [ ]:
MODEL_NAME = "cl-tohoku/bert-base-japanese-whole-word-masking"
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", max_length=128, truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["text"])

train_dataset = tokenized_datasets["train"].shuffle().select(range(4000))
eval_dataset = tokenized_datasets["test"].shuffle().select(range(256))

# Save dataset
train_dataset.save_to_disk("./train/")
eval_dataset.save_to_disk("./test/")

実際にどのように変換されているのか、以下のスクリプトを実行し確認してみましょう。

In [ ]:
index = 50000
print(dataset["train"][index])
print('Tokenize:', tokenizer.tokenize(dataset["train"]['text'][index]))
print('Encode:', tokenizer.encode(dataset["train"]['text'][index]))

## Trainer API を使用した トレーニング（ファインチューニング）実行
Transformers には Trainer という便利なクラスがあり、Torch Neuron からも利用可能です。 ここでは Trainer API を利用してトレーニングを実行していきます。

- With transformers==4.44.0, running one worker fine-tuning without torchrun would result in a hang. To workaround and run one worker fine-tuning, use torchrun --nproc_per_node=1 <script>.

### neuron_parallel_compile による事前コンパイル
トレーニングの各ステップでは、グラフがトレースされ、トレースされたグラフが以前のものと異なる場合は、再度計算グラフのコンパイルが発生します。大規模なモデルの場合、各グラフのコンパイル時間が長くなることがあり、トレーニング時間の中で占めるコンパイル時間がボトルネックとなってしまう場合もあり得ます。このコンパイル時間を短縮するため、PyTorch Neuron では neuron_parallel_compile ユーティリティが提供されています。neuron_parallel_compile は、スクリプトの試行からグラフを抽出し並列事前コンパイルを実施、コンパイル結果（NEFF : Neuron Executable File Format）をキャッシュとしてディスク上に保持します。

では実際に事前コンパイルを実行してみましょう。以下の内容でbert-jp-precompile.pyというファイル名の Python スクリプトを作成し実行します。スクリプトは基本的にこの後実行するトレーニングスクリプトと同じ内容ですが、neuron_parallel_compileはグラフの高速コンパイルのみを目的とし実際の演算は実行されず、出力結果は無効となります。トレーニング実行中も必要に応じてグラフはコンパイルされるため、この事前コンパイルのプロセスはスキップし、次のトレーニング処理に直接進んでも問題はありません。

コンパイル時間を短縮するためデータセット、epoch 数を制限している点にご注意ください。コンパイル結果は `/var/tmp/neuron-compile-cache/` 以下に保存されています。

In [ ]:
%%writefile bert-jp-precompile.py

from transformers import BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_from_disk
import torch, torch_xla.core.xla_model as xm
import os

os.environ["NEURON_CC_FLAGS"] = "--model-type=transformer"

device = xm.xla_device()

MODEL_NAME = "cl-tohoku/bert-base-japanese-whole-word-masking"
model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2).to(device)

train_dataset = load_from_disk("./train/").with_format("torch")
train_dataset = train_dataset.select(range(64))

eval_dataset = load_from_disk("./test/").with_format("torch")
eval_dataset = eval_dataset.select(range(64))

training_args = TrainingArguments(
    num_train_epochs = 2,
    learning_rate = 5e-5,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    output_dir = "./results",
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
)

train_result = trainer.train()

In [ ]:
!time XLA_USE_BF16=1 neuron_parallel_compile torchrun --nproc_per_node=1 bert-jp-precompile.py

### シングルワーカーでのトレーニング実行
次に実際にトレーニングを実行してみます。事前コンパイルを実行した場合でも、追加のコンパイルが発生することがあります。一通りのコンパイルが終了した後、2度目以降の実行では、Neuron コアの恩恵を受けた高速トレーニングを体験できます。以下の内容で bert-jp-single.py というファイル名の Python スクリプトを作成し実行してみましょう。

先程の事前コンパイルとは異なり、今回は実際にトレーニングを実行するため、用意したデータセット全てに対して epoch = 10 で実行します。

In [ ]:
%%writefile bert-jp-single.py

from transformers import BertForSequenceClassification, BertJapaneseTokenizer, Trainer, TrainingArguments
from datasets import load_from_disk
import torch, torch_xla.core.xla_model as xm
import os

os.environ["NEURON_CC_FLAGS"] = "--model-type=transformer"

device = xm.xla_device()

MODEL_NAME = "cl-tohoku/bert-base-japanese-whole-word-masking"
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)
model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2).to(device)

train_dataset = load_from_disk("./train/").with_format("torch")
eval_dataset = load_from_disk("./test/").with_format("torch")

training_args = TrainingArguments(
    num_train_epochs = 10,
    learning_rate = 5e-5,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    output_dir = "./results",
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    tokenizer = tokenizer,
)

train_result = trainer.train()
print(train_result)

eval_result = trainer.evaluate()
print(eval_result)

trainer.save_model("./results")

In [ ]:
!time XLA_USE_BF16=1 torchrun --nproc_per_node=1 bert-jp-single.py

ステップ数 5000 のトレーニングが 9~10分程で完了しました。

トレーニング実行中に、AWS Neuron で提供される `neuron-top` ツールを利用すると、Neuron コア及び vCPU の利用率、アクセラレータメモリ、ホストメモリの利用状況等を確認することができます。inf2.xlarge には、一つの Inferentia2 チップ、チップ内に二つの Neuron コアが搭載されています。結果から、二つある Neuron コア（NC0 及び NC1）のうち一つの Neuron コアのみが利用されていることが分かります。まだ最適化の余地はありそうです。

生成されたモデルから期待通りの出力が得られるか確認しておきましょう。

In [ ]:
from transformers import pipeline

classifier = pipeline("text-classification", model = "./results/")

print(classifier("大変すばらしい商品でした。感激です。"))
print(classifier("期待していた商品とは異なりました。残念です。"))

期待通りの出力を得られることが確認できたようです。

## torchrun を用いたマルチワーカーでのトレーニング実行
それでは、先程のトレーニングスクリプトに変更を加え、二つある Neuron コアを有効活用してみましょう。複数の Neuron コアを利用したマルチワーカーで実行するためには `torchrun` コマンドを利用します。`torchrun` コマンドに対して、オプション `--nproc_per_node` で利用する Neuron コアの数（並列実行するワーカー数）を指定します。trn1.2xlarge (inf2.xlarge) では 2 を、trn1.32xlargeの場合は 2, 8, 32 が指定可能です。

`torchrun` を利用したデータパラレルトレーニングを実行するにあたって、先程のスクリプトに一部変更を加えた `bert-jp-dual.py` というファイル名のスクリプトを作成し実行します。

それでは変更後のスクリプトを利用して　inf2.xlarge 上の二つ Neuron コアを利用したトレーニングを実行してみましょう。シングルワーカーでのトレーニング結果と比較し `Total train batch size` の値が倍の 16 に、`Total optimization steps` が半分の 2500 となっている点を確認できると思います。

シングルワーカー時の手順同様、まずは事前コンパイルを実行し、その後に実際のトレーニングを実行します。

In [ ]:
%%writefile bert-jp-dual-precompile.py

from transformers import BertForSequenceClassification, BertJapaneseTokenizer, Trainer, TrainingArguments
from datasets import load_from_disk
import torch, torch_xla.distributed.xla_backend
import os

os.environ["NEURON_CC_FLAGS"] = "--model-type=transformer"

MODEL_NAME = "cl-tohoku/bert-base-japanese-whole-word-masking"
model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

train_dataset = load_from_disk("./train/").with_format("torch")
train_dataset = train_dataset.select(range(64))

eval_dataset = load_from_disk("./test/").with_format("torch")
eval_dataset = eval_dataset.select(range(64))


training_args = TrainingArguments(
    num_train_epochs = 2,
    learning_rate = 5e-5,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    output_dir = "./results",
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
)

train_result = trainer.train()
print(train_result)

eval_result = trainer.evaluate()
print(eval_result)

In [ ]:
!time XLA_USE_BF16=1 neuron_parallel_compile torchrun --nproc_per_node=2 bert-jp-dual-precompile.py

In [ ]:
%%writefile bert-jp-dual.py

from transformers import BertForSequenceClassification, BertJapaneseTokenizer, Trainer, TrainingArguments
from datasets import load_from_disk
import torch, torch_xla.distributed.xla_backend
import os

os.environ["NEURON_CC_FLAGS"] = "--model-type=transformer"

MODEL_NAME = "cl-tohoku/bert-base-japanese-whole-word-masking"
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)
model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

train_dataset = load_from_disk("./train/").with_format("torch")
eval_dataset = load_from_disk("./test/").with_format("torch")

training_args = TrainingArguments(
    num_train_epochs = 10,
    learning_rate = 5e-5,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    output_dir = "./results",
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    tokenizer = tokenizer,
)

train_result = trainer.train()
print(train_result)

eval_result = trainer.evaluate()
print(eval_result)

trainer.save_model("./results")

In [ ]:
!time XLA_USE_BF16=1 NEURONCORE_NUM_DEVICES=2 torchrun --nproc_per_node=2 bert-jp-dual.py

トレーニング実行中の neuron-top の出力も確認してみましょう。今度は二つの Neuron コアが利用されている事が確認できると思います。トレーニングに要する実行時間も 5~6 分に削減されました。

## 推論実行
先ほどは生成されたモデルから期待通りの出力が得られるかどうかCPU上で推論実行し、結果を確認しました。ここでは生成されたモデルをinf2.xlarge上で推論実行します。

In [ ]:
import torch
import torch_neuronx
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import transformers

from transformers import BertForSequenceClassification, BertJapaneseTokenizer

def encode(tokenizer, *inputs, max_length=128, batch_size=1):
    tokens = tokenizer.encode_plus(
        *inputs,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors="pt"
    )
    return (
        torch.repeat_interleave(tokens['input_ids'], batch_size, 0),
        torch.repeat_interleave(tokens['attention_mask'], batch_size, 0),
        torch.repeat_interleave(tokens['token_type_ids'], batch_size, 0),
    )

tokenizer = AutoTokenizer.from_pretrained("./results/")
model = AutoModelForSequenceClassification.from_pretrained("./results/", torchscript=True)


sequence = "大変すばらしい商品でした。感激です。"
paraphrase = encode(tokenizer, sequence)
cpu_paraphrase_logits = model(*paraphrase)[0]
print('CPU paraphrase logits:', cpu_paraphrase_logits.detach().numpy())

sequence = "期待していた商品とは異なりました。残念です。"
paraphrase = encode(tokenizer, sequence)
cpu_paraphrase_logits = model(*paraphrase)[0]
print('CPU paraphrase logits:', cpu_paraphrase_logits.detach().numpy())

### Compile the model for Neuron

In [ ]:
filename = 'model.pt'

In [ ]:
model_neuron = torch_neuronx.trace(model, paraphrase)

# Save the TorchScript for inference deployment
torch.jit.save(model_neuron, filename)

### Run inference and compare results

In [ ]:
model_neuron = torch.jit.load(filename)

sequence = "大変すばらしい商品でした。感激です。"
paraphrase = encode(tokenizer, sequence)
neuron_paraphrase_logits = model_neuron(*paraphrase)[0]
print('Neuron paraphrase logits:', neuron_paraphrase_logits.detach().numpy())

sequence = "期待していた商品とは異なりました。残念です。"
paraphrase = encode(tokenizer, sequence)
neuron_paraphrase_logits = model_neuron(*paraphrase)[0]
print('Neuron paraphrase logits:', neuron_paraphrase_logits.detach().numpy())

CPUで推論実行した結果と同様の結果が得られている事が確認できました。推論性能を評価する方法は以下のサンプルをご参照下さい。
+ https://github.com/aws-neuron/aws-neuron-sdk/blob/master/src/examples/pytorch/torch-neuronx/bert-base-cased-finetuned-mrpc-inference-on-trn1-tutorial.ipynb